# 5.6.1 Affine 계층

np.dot으로 행렬의 곱셈을 표현했던 것을 기억해보자.

In [5]:
import numpy as np

X = np.random.rand(2)    # input
W = np.random.rand(2, 3) # weight
B = np.random.rand(3)    # bias

print(X.shape, '-> 1 x 2')
print(W.shape, '-> 2 x 3')
print(B.shape, '-> 1 x 3')

np.dot(X, W) + B

(2,) -> 1 x 2
(2, 3) -> 2 x 3
(3,) -> 1 x 3


array([0.42270627, 0.78388733, 1.25228132])

행렬의 곱은 아래와 같은 특징이 있다.

* 행렬곱을 하기 위해서는 앞/뒤의 차원 수가 같아야 한다.
* 행렬의 곱을 기하학에서는 Affine 변환이라고 한다.

이 Affine 변환 역시 계산 그래프로 표현 가능하다.


<p align="center"><img src="imgs/5-24.png" width=600></p>

왜 이제와서 affine이 나오는가? 지금까지의 계산은 스칼라값이 흘렀는데, 이제 행렬이 흐르고 있기 때문이다. 

* 아래 예시에서는 편의상 backward에 x를 그대로 넣었다
* 하지만 실제 계산에서는, global loss로부터 계산된 배열이 들어간다.
* 그러니 값들은 그냥 참고용으로만 알아두자.

좋은 자료: 오차 역전파를 계산그래프로 설명하는 유튜브 영상
https://www.youtube.com/watch?v=1Q_etC_GHHk&t=606s

# 5.6.2 배치용 Affine 계층

In [9]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print('X_dot_W: \n', X_dot_W)
print('B:', B)

X_dot_W: 
 [[ 0  0  0]
 [10 10 10]]
B: [1 2 3]


In [10]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

위와 같은 현상이 일어나는 것은 numpy가 가지고 있는 broadcasting 기능 덕분이다. 이는 shape가 작은 행렬이 더 큰 행렬에 맞춰서 알아서 크기가 조절되는 것을 말한다. (확장)

In [11]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
dB = np.sum(dY, axis = 0)
dB

array([5, 7, 9])

Affine 계층의 bias 계산되는 원리를 이해하면 된다.

In [25]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

In [30]:
W = np.array([[1, 2, 3], [4, 5, 6]])
B = np.array([4, 4, 4])
a = Affine(W, B)

X = np.array([1.0, 2.0])
a.forward(X)
print(a.original_x_shape)

(2,)


# 5.6.3 Softmax-with-Loss 계층

소프트맥스 함수는 입력 값을 정규화하여 출력한다. 예를 들어 손글씨 숫자 인식에서의 softmax 계층 출력은 아래 그림 처럼 된다. 3번째 확률값이 제일 큰 것에 주목!

<p align="center"><img src="imgs/5-28.png" width=600></p>

이제 softmax와 cross-entropy를 합친 softmax-with-loss라는 클래스를 구현해볼 것이다. 2개를 합친 것을 그림으로 나타내면 아래와 같다.

<p align="center"><img src="imgs/5-29.png" width=600></p>

뭐가 굉장히 복잡해 보이는데, 사실 softmax와 cross-entropy를 붙여놓은 것이라 크게 어렵지는 않다. 이걸 간소화 하면 밑에 처럼 표현할 수 있다.

<p align="center"><img src="imgs/5-30.png" width=600></p>

여기서 y는 softmax의 출력값이고, t는 원래 라벨이다. 앞에서 설명한건 시그모이드고, 이 책의 어디에서도 softmax와 cross entropy의 계산그래프에 대해 설명하지 않았다. 그러니 아래 포인트 정도만 이해하고 다음으로 넘어가면 된다. 궁금한 사람들은 각자 부록을 공부하면 된다!

* input과 output에만 주목하자. 결국 미분의 output은 y-t가 되는데, 원래 라벨에 대한 정보가 담겨 있다.
* 라벨과 예측한 값에 대한 오차가 크면? -> 많이 움직여야 한다.
* 역전파의 결과가 상당히 깔끔한데? -> 원래 그렇게 설계되었기 때문이다.

이를 코드로 구현해보면 아래와 같이 된다.

In [1]:
from common import layers

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # 소프트맥스 출력
        self.x = None # 정답 라벨

    def forward(self, x, t):
        self.t = t
        self.y = layers.softmax(x)
        self.loss = layers.cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # 배치사이즈로 나눠주는 이유? -> 이미지 1개당 오차를 전달해야 하니까
        return dx